# E6a Vector memory lets you manipulate information in a new kind of way

## A quick business review (because the technology alone is not enough)

Every business benefits from four courses of action to be used together or in combination:

1. Grow the existing business
2. Save money and time
3. Add completely new business
4. Prepare for the unknown

![](assets/shopkeeper.png)

The business owner is always leaking time and money from their valuable buckets of time and money. Let's help them succeed!

## 🔥 Get a kernel ready using our vector memory store

We're going to create a kernel that uses our new vector memory store that's flush with information we can start using.

In [1]:
import semantic_kernel as sk
from IPython.display import display, Markdown

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
    kernel.add_text_embedding_generation_service("ada", AzureTextEmbedding("text-embedding-ada-002", api_key, endpoint))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
#    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-4", api_key, org_id))
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
    kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=ChromaMemoryStore(persist_directory='mymemories'))

memoryCollectionName = "SWOT"

/Users/johnmaeda/Documents/GitHub/pizzashop-and-ai/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 🔖 Recall two of the business' strengths in the SWOT

In our early SWOT analysis, we distilled two possible differentiated strengths with this pizza store.

| STRENGTH 1 | STRENGTH 2 |
|---|---|
| The best garlic pizza in the business | The store owner was trained in Sicily |
| ![](assets/garlicpizza.png) | ![](assets/measuring.png) |

We take our original tool and modify it to drive a `ResearchInterview` semantic function that leverages our vat of context.

In [2]:
topic = "The business makes the best garlic pizza and the owner was trained in Sicily."
swot_quality = "Strengths"
counter = 0

gathered_context = []
max_memories = 3
memories = await kernel.memory.search_async(memoryCollectionName, topic, limit=max_memories, min_relevance_score=0.75)

print(f"✨ Leveraging information available to address '{topic}'...")

for memory in memories:
    if counter == 0:
        related_memory = memory.text
    counter += 1
    gathered_context.append(memory.text + "\n")
    print(f"  > 😶‍🌫️ Hit {counter}: {memory.id} ")

pluginDirectory = "./plugins-sk"
print(f"✨ Synthesizing question list suitable for a research interview...")
pluginFC = kernel.import_semantic_skill_from_directory(pluginDirectory, "FriendlyConsultant");

my_context = kernel.create_new_context()
my_context['input'] = topic
my_context['context'] = "\n".join(gathered_context)
my_context['swot_quality'] = swot_quality

interview_result = await kernel.run_async(pluginFC["ResearchInterview"], input_context=my_context)

display(Markdown(f"## ✨ Generated list of questions to address '{topic}'...\n"+str(interview_result)))


✨ Leveraging information available to address 'The business makes the best garlic pizza and the owner was trained in Sicily.'...
  > 😶‍🌫️ Hit 1: costefficiency 
  > 😶‍🌫️ Hit 2: strength-1 
  > 😶‍🌫️ Hit 3: Recipe and process for making the signature garlic pizza 
✨ Synthesizing question list suitable for a research interview...


## ✨ Generated list of questions to address 'The business makes the best garlic pizza and the owner was trained in Sicily.'...
1. Can you provide more information about the owner's training in Sicily? How has this training influenced the business's operations and success?
2. How did the business develop its unique garlic pizza recipe? Has it undergone any changes or improvements over time?
3. Can you provide more information about the staff turnover rate? What steps has the business taken to address this issue?
4. How does the business currently train new staff members? Are there any areas that could be improved or streamlined?
5. How does the business currently gather customer feedback? Has it identified any areas for improvement or expansion in its menu or operations?
6. Can you provide more information about the negative reviews from younger demographics? How has the business responded to these reviews in the past?
7. How does the business currently manage its online reputation? Are there any areas that could be improved or expanded upon?
8. How does the business currently measure customer satisfaction? Are there any areas that consistently receive high or low ratings?

## 📄 With the list of questions in hand, let's get answers

We'll use the `Presentation` semantic function to generate a logical "idea pitch" or "solution pitch" to tackle a business challenge.

In [6]:
interview_transcript="""
Q: What specific training did the owner receive in Sicily and how has it been applied to the business?
A: The owner learned how to use a special kind of brick oven to make pizzas in Sicily.

Q: How has the unique garlic pizza recipe contributed to the success of the business?
A: Over half of the pizzas sold are the signature garlic pizza.

Q: What awards has the garlic pizza recipe won and how have these awards impacted the business?
A: The owner won the "best pizza of 2022" award from the local newspaper.

Q: How does the business currently utilize the owner's training and the unique garlic pizza recipe to its advantage?
A: The owner trains all new employees on how to make the garlic pizza.

Q: What feedback have customers given about the garlic pizza and the overall service of the business?
A: Customers come back for the garlic pizza.

Q: How does the business maintain the quality and consistency of the garlic pizza?
A: The owner has a simple recipe of flour, water, eggs, and garlic in specific and repeatable proportions.

Q: How does the business ensure that the skills and knowledge of the owner are passed on to the staff?
A: Right now it is all in the owner's head and shared verbally.

Q: What steps has the business taken to protect the unique garlic pizza recipe?
A: The recipe requires a special blend of cheeses that only the owner knows.

Q: How does the business plan to continue leveraging the owner's training and the unique garlic pizza recipe in the future?
A: The owner is looking for an apprentice to train.

Q: How does the business differentiate itself from competitors with similar offerings?
A: The garlic pizza is unique to this business.

Q: How does the business plan to address any potential weaknesses or threats that could impact the owner's training or the unique garlic pizza recipe?
A: Currently there's no plan to do so.
"""

skillsDirectory = "./plugins-sk"

topic = "How to protect the uniqueness of the garlic pizza recipe"
counter = 0
gathered_context = []
max_memories = 3
memories = await kernel.memory.search_async(memoryCollectionName, topic, limit=max_memories, min_relevance_score=0.75)
pluginFC = kernel.import_semantic_skill_from_directory(skillsDirectory, "FriendlyConsultant");

print(f"🤖 Leveraging information available to address '{topic}'...")

for memory in memories:
    if counter == 0:
        related_memory = memory.text
    counter += 1
    gathered_context.append(memory.text + "\n")
    print(f"  > 😶‍🌫️ Hit {counter}: {memory.id} ")

my_context = kernel.create_new_context()
my_context['input'] = topic
my_context['context'] = "\n".join(gathered_context) + "\n" + interview_transcript

preso_result = await kernel.run_async(pluginFC["Presentation"], input_context=my_context)

print(f"😶‍🌫️ New context for '{topic}' appended to vector database...")
await kernel.memory.save_information_async(memoryCollectionName, id=f"{topic}", text=topic + "\n\n" + str(preso_result))

display(Markdown("# ✨ Generated presentation...\n"+str(preso_result)))


🤖 Leveraging information available to address 'How to protect the uniqueness of the garlic pizza recipe'...
  > 😶‍🌫️ Hit 1: costefficiency 
  > 😶‍🌫️ Hit 2: timeefficiency 
  > 😶‍🌫️ Hit 3: strength-0 
😶‍🌫️ New context for 'How to protect the uniqueness of the garlic pizza recipe' appended to vector database...


# 🤖 Generated ...
---
# Protecting the Uniqueness of the Garlic Pizza Recipe

This presentation will provide suggestions for how to protect the uniqueness of the garlic pizza recipe for a business striving to improve itself. 

---
## The Question

How can the business protect the uniqueness of the garlic pizza recipe?

---
## Key Concerns

1. Maintaining the secret recipe
2. Preventing recipe theft
3. Protecting against competitors copying the recipe

---
## Concern 1: Maintaining the Secret Recipe

**Explanation:** The unique garlic pizza recipe is a key strength of the business, and it is important to maintain the secret recipe to protect the business's competitive advantage.

**Example:** The business can limit access to the recipe by only allowing the owner to prepare the garlic pizza. The recipe can be stored in a secure location and only shared with trusted employees who have signed a non-disclosure agreement.

---
## Concern 2: Preventing Recipe Theft

**Explanation:** Recipe theft is a common issue in the food industry, and it is important to take steps to prevent the recipe from being stolen.

**Example:** The business can implement security measures such as surveillance cameras and access controls to prevent unauthorized access to the recipe. The recipe can also be trademarked or patented to provide legal protection.

---
## Concern 3: Protecting Against Competitors Copying the Recipe

**Explanation:** Competitors may try to copy the unique garlic pizza recipe, which can impact the business's competitive advantage.

**Example:** The business can differentiate itself by emphasizing the unique aspects of the garlic pizza recipe, such as the owner's training in Sicily and the special blend of cheeses used. The business can also monitor competitors and take legal action if necessary to protect the recipe.

---
## Summary

To protect the uniqueness of the garlic pizza recipe, the business should maintain the secret recipe, prevent recipe theft, and protect against competitors copying the recipe. By taking these steps, the business can maintain its competitive advantage and continue to provide customers with a unique and delicious pizza experience.

## 🍎 Training the trainers: Making it easy and painless

Recognizing the importance of the garlic pizza, it makes sense for the owner to get the recipe out of their head and into a training program. The owner does the work of writing down his expert tips for making the signature garlic pizza dish.

In [7]:
what_is_this = "Recipe and process for making the signature garlic pizza"
expert_knowledge = """
Special Garlic Pizza Recipe:
> Ingredients: flour, eggs, sugar, garlic, salt, yeast, water, olive oil, tomato sauce, mozzarella cheese, parmesan cheese, basil

Special Cheese Blend Recipe:
> Ingredients: mozzarella cheese, parmesan cheese, cheddar cheese

Special Cooking Instructions:
> 1. Mix flour, eggs, sugar, garlic, salt, yeast, water, and olive oil in a bowl
> 2. Knead the dough
> 3. Let the dough rise
> 4. Roll out the dough
> 5. Spread tomato sauce on the dough
> 6. Sprinkle cheese blend and basil on the dough
> 7. Bake the pizza in the brick oven at 350 degrees for 10 minutes
> 8. Be sure to move the pizza around within the oven every 3 minutes
"""

print(f"😶‍🌫️ New context for '{what_is_this}' appended to vector database...")
await kernel.memory.save_information_async(memoryCollectionName, id=f"{what_is_this}", text=what_is_this + "\n\n" + expert_knowledge)

😶‍🌫️ New context for 'Recipe and process for making the signature garlic pizza' appended to vector database...


To turn this information into a simple training program, we turn to the `TrainingProgram` semantic function to sketch a possible approach.

In [8]:
pluginsDirectory = "./plugins-sk"

topic = "How to make the signature garlic pizza and save time"
counter = 0
gathered_context = []
max_memories = 5
memories = await kernel.memory.search_async(memoryCollectionName, topic, limit=max_memories, min_relevance_score=0.75)
pluginFC = kernel.import_semantic_skill_from_directory(pluginsDirectory, "FriendlyConsultant");

print(f"🤖 Leveraging information available to address '{topic}'...")

for memory in memories:
    if counter == 0:
        related_memory = memory.text
    counter += 1
    gathered_context.append(memory.text + "\n")
    print(f"  > 😶‍🌫️ Hit {counter}: {memory.id} ")

my_context = kernel.create_new_context()
my_context['input'] = topic
my_context['context'] = "\n".join(gathered_context)

training_result = await kernel.run_async(pluginFC["TrainingProgram"], input_context=my_context)

display(Markdown("# ✨ Generated ...\n"+str(training_result)))

🤖 Leveraging information available to address 'How to make the signature garlic pizza and save time'...
  > 😶‍🌫️ Hit 1: Recipe and process for making the signature garlic pizza 
  > 😶‍🌫️ Hit 2: costefficiency 
  > 😶‍🌫️ Hit 3: How to protect the uniqueness of the garlic pizza recipe 
  > 😶‍🌫️ Hit 4: timeefficiency 
  > 😶‍🌫️ Hit 5: strength-0 


# ✨ Generated ...
How to Make the Signature Garlic Pizza and Save Time

Learning Objectives:
1. Understand the importance of making the signature garlic pizza efficiently
2. Learn the specific steps and procedures for making the garlic pizza
3. Encourage participants to ask questions and seek help when needed

Agenda:
1. Introduction (5 minutes)
- Welcome participants and explain the purpose of the training program
- Explain the importance of making the signature garlic pizza efficiently

2. Recipe and Process Overview (20 minutes)
- Review the special garlic pizza recipe and special cheese blend recipe
- Explain the special cooking instructions step-by-step
- Demonstrate the process for making the garlic pizza

3. Time-Saving Tips (20 minutes)
- Identify areas where time can be saved in the process
- Demonstrate techniques for streamlining the process, such as prepping ingredients in advance and organizing workstations
- Encourage participants to share their own time-saving tips

4. Q&A and Discussion (10 minutes)
- Encourage participants to ask questions and seek help when needed
- Discuss any challenges or issues that participants may have encountered when making the garlic pizza

5. Conclusion (5 minutes)
- Summarize the key takeaways from the training program
- Encourage participants to apply what they have learned when making the garlic pizza in the future

Total Time: 1 hour